In [1]:
import torch
from diffusers import StableDiffusionPipeline
from diffusers.utils import export_to_gif
import os
import os.path as path

home = os.getenv('HOME')
p = StableDiffusionPipeline.from_single_file(path.join(home, 'civitai', 'animatediff', 'toonyou_beta3.safetensors'))

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
/home/kiddos/.pyenv/versions/3.11.4/lib/python3.11/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [2]:
import torch
from diffusers import MotionAdapter, AnimateDiffPipeline, DDIMScheduler
from diffusers.utils import export_to_gif
import os
import os.path as path

adapter = MotionAdapter.from_pretrained("guoyww/animatediff-motion-adapter-v1-5-2")
model_id = "SG161222/Realistic_Vision_V5.1_noVAE"
pipe = AnimateDiffPipeline.from_pretrained(
  model_id,
  vae=p.vae.to(torch.float16),
  unet=p.unet.to(torch.float16),
  tokenizer=p.tokenizer,
  motion_adapter=adapter,
  torch_dtype=torch.float16,
  use_safetensors=True,
)
scheduler = DDIMScheduler.from_pretrained(
  model_id,
  subfolder="scheduler",
  clip_sample=False,
  timestep_spacing="linspace",
  beta_schedule="linear",
  steps_offset=1,
)
pipe.scheduler = scheduler

# pipe.enable_sequential_cpu_offload()
pipe.enable_vae_slicing()
pipe.enable_model_cpu_offload()

The config attributes {'motion_activation_fn': 'geglu', 'motion_attention_bias': False, 'motion_cross_attention_dim': None} were passed to MotionAdapter, but are not expected and will be ignored. Please verify your config.json configuration file.


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

The config attributes {'transformer_layers_per_block': 1, 'attention_head_dim': 8, 'class_embed_type': None, 'addition_embed_type': None, 'addition_time_embed_dim': None, 'upcast_attention': None, 'projection_class_embeddings_input_dim': None} were passed to UNetMotionModel, but are not expected and will be ignored. Please verify your config.json configuration file.


In [3]:
prompt = '(masterpiece, best quality), 1girl, collarbone, wavy hair, looking at viewer, blurry foreground, upper body, necklace, contemporary, plain pants, ((intricate, print, pattern)), ponytail, freckles, red hair, dappled sunlight, smile, happy,'
negative_prompt = '(worst quality, low quality, letterboxed)'

generator = torch.Generator("cpu").manual_seed(42019273098)
output = pipe(
  prompt=prompt.strip(),
  negative_prompt=negative_prompt.strip(),
  num_frames=16,
  guidance_scale=7.5,
  num_inference_steps=30,
  generator=generator,
)
frames = output.frames[0]
export_to_gif(frames, "/tmp/animatediff.gif")

  0%|          | 0/30 [00:00<?, ?it/s]

'/tmp/animatediff.gif'